[Node 14: ctypes](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node14.html)

Navigation:

**Next:** [SWIG](node15.ipynb) **Up:** [Python und C/C++](node13.ipynb) **Previous:** [Python und C/C++](node13.ipynb)

##  ctypes
Mit Hilfe des Python-Standardmoduls [``ctypes``](https://docs.python.org/3/library/ctypes.html)  können Funktionen aus dynamischen Bibliotheken ohne Erweiterungsmodule aufgerufen werden. Der Datentypen-Unterschied zwischen Python und C wird durch einige   ``c_*``-Datentypen überbrückt, z.B.  ``c_short: int/long``,  ``c_double: double``,  ``c_char_p: str``. 

Auch Pointer können verarbeitet werden:

In [ ]:
from ctypes import *
i = c_int()
s = c_wchar_p('Hello World!') # wchar_t *
type(s)

In [ ]:
s.value

In [ ]:
i.value

In [ ]:
i  = c_int(42)
pi = pointer(i)
pi

In [ ]:
pi.contents.value

 ``ctypes``  wird verwendet, um Funktionen aus dynamischen C-Bibliotheken zu laden und aufzurufen. Zunächst muss die Biliothek geladen werden, um dann die C-Funktionen aufzurufen:

In [ ]:
from ctypes import *
libc=cdll.LoadLibrary('libc.so.6')
libc

In [ ]:
dir(libc)

 Die C-Funktionen  ``printf``  oder  ``sleep``  können dynamisch geladen werden:

In [ ]:
libc.printf

In [ ]:
libc.sleep

In [ ]:
[s for s in dir(libc) if s[:2] != '__' and s[-2:] != '__']

(Diese Liste umfasst nicht alle Funktionen, sondern nur diejenigen, die wir vorher angesprochen haben. Eine komplette Liste aller definierten Funktionen kann mit `nm -D` ausgegeben werden:

In [ ]:
!nm -D `locate libc.so.6 | head -n1`

 Argument- und Rückgabe-Werte sollten mit  ``argtypes``  und  ``restypes``   definiert werden:

In [ ]:
from ctypes import *
libm=cdll.LoadLibrary('libm.so.6')
cos=libm.cos
cos

In [ ]:
# throws an error:
cos(3.14159265)

In [ ]:
# returns wrong result
cos(c_double(3.14159265))

In [ ]:
# returns wrong result
cos(c_double(0.0))

In [ ]:
# returns wrong result
cos(c_double(0.1))

In [ ]:
cos.argtypes=[c_double]
cos.restype=c_double
# now it works
cos(c_double(0.0))

In [ ]:
cos(3.14159265)

 Funktionen, die in einen Puffer schreiben, werden folgendermaßen aufgerufen (Beispiel hier: [gethostname](http://man7.org/linux/man-pages/man2/gethostname.2.html)):

In [ ]:
from ctypes import *
libc = cdll.LoadLibrary('libc.so.6')
gethostname = libc.gethostname
gethostname.argtypes = [c_char * 255, c_uint]
gethostname.restype  = c_int
# create the char buffer and call the function
buf = create_string_buffer(255)
gethostname(buf, 5) # too short to hold hostname

In [ ]:
buf.value # is truncated

In [ ]:
gethostname(buf, 30) # more space (our buffer has actually 255 chars)

In [ ]:
buf.value